# TD-TME10: résolution de problèmes

L'objectif de ce TP est d'implémenter des algorithmes de résolution de problèmes.

Dans un premier temps, on va utiliser le jeu du taquin $3\times 3$ présenté en cours comme problème à résoudre. Pensez-donc à regarder la description du jeu dans les transparents du cours 10.

## Etape 1: Modélisation d'un graphe d'états pour le jeu du Taquin

Afin de pouvoir gérer des piles ou des files d'états, on utilise la librairie *collections* de Python et l'ensemble des fonctions de gestion des listes *deque*.

Familiarisez-vous avec cette librairie en regardant la documentation en ligne de votre version de Python et en testant les instructions suivantes.

In [1]:
from collections import deque
# Déclaration
fileEtat = deque()

# Remplissage à partir d'une chaîne (par exemple)
for i in 'abcde':
    fileEtat.append(i)
print("Contenu de fileEtat: ",fileEtat)

# Utilisation comme une pile: LIFO (Last In First Out)
print("Récupération LIFO des éléments:")
while len(fileEtat) > 0:
    a= fileEtat.pop()
    print(a)
print("Contenu de fileEtat: ",fileEtat)

# Remplissage à partir d'une chaîne (par exemple)
for i in 'abcde':
    fileEtat.append(i)
print("Contenu de fileEtat: ",fileEtat)

# Utilisation comme une file: FIFO (First In First Out)
print("Récupération FIFO des éléments:")
while len(fileEtat) > 0:
    print(fileEtat.popleft())
print(fileEtat)
    
    

('Contenu de fileEtat: ', deque(['a', 'b', 'c', 'd', 'e']))
Récupération LIFO des éléments:
e
d
c
b
a
('Contenu de fileEtat: ', deque([]))
('Contenu de fileEtat: ', deque(['a', 'b', 'c', 'd', 'e']))
Récupération FIFO des éléments:
a
b
c
d
e
deque([])


### Représentation du damier du Taquin
Le Taquin est représenté sous la forme d'une liste de valeurs entières entre 0 et 9. La valeur 0 représente la case vide.

Pour pouvoir gérer proprement un Taquin, on encapsule cette liste dans la classe suivante.

In [2]:
class Taquin:
    def __init__(self,grille):
        """ grille: liste de 9 entiers de 0 à 9
            Hypothese: la grille donnée est correcte (il faudrait faire une vérification...)
        """
        self.grille = grille
    
    def estFinal(self):
        """ rend True si c'est une position finale du Taquin"""
        reference = [1,2,3,8,0,4,7,6,5]
        return self.grille == reference
    
    def estCoupValide(self,coup):
        """ coup: entier de 1 à 9
            -> pour vérifier qu'un coup peut être jouer avant de le jouer
        """
        if coup == 0:
            # on ne joue pas la case vide !
            return False
        # le coup est valide si on peut déplacer la valeur vers la case 0
        indexCoup = self.grille.index(coup)
        indexZero = self.grille.index(0)
        # On vérifie si les cases sont à côté
        coordCoup = (int(indexCoup / 3.0), indexCoup % 3)  # (numLigne, numColonne)
        coordZero = (int(indexZero / 3.0), indexZero % 3)  # (numLigne, numColonne)
        if (coordCoup[0] == coordZero[0]):  
            # déplacement sur même ligne
            return (coordCoup[1] == coordZero[1]+1) or (coordCoup[1] == coordZero[1]-1)
        elif (coordCoup[1] == coordZero[1]): 
            # déplacement sur une même colonne
            return (coordCoup[0] == coordZero[0]+1) or (coordCoup[0] == coordZero[0]-1)
        
    def tousLesCoupsJouables(self):
        """ rend la liste des jouables (coups valides donc) dans la position 
            de ce taquin
        """
        return [i for i in range(1,9) if self.estCoupValide(i)]
                    
    def joueLeCoup(self,coup):
        """ coup: entier de 1 à 9 qui correspond à la valeur faut déplacer donc
            Hypothese: le coup est valide
            -> pour jouer le coup on échange l'entier donné avec le chiffre 0 (case vide)
            La fonction renvoie un nouveau Taquin (le Taquin courant n'est donc pas modifié)
        """
        indexCoup = self.grille.index(coup)
        coordCoup = (int(indexCoup / 3.0), indexCoup % 3)  # (numLigne, numColonne)
        indexZero = self.grille.index(0)
        coordZero = (int(indexZero / 3.0), indexZero % 3)  # (numLigne, numColonne)
        resultat = []
        for i in range(0,len(self.grille)):
            if i == indexCoup:
                resultat.append(0)
            elif i == indexZero:
                resultat.append(coup)
            else:
                resultat.append(self.grille[i])
        grilleResultat = Taquin(resultat)
        return grilleResultat
        
    def toString(self):
        """ Conversion de la grille en une chaîne compacte
            Cette chaîne sert à pouvoir mettre des taquins dans un set() ou un dict()
        """
        return ''.join(str(e) for e in self.grille)
        
    def affiche(self):
        """ Affichage simple du taquin """
        print("Grille du taquin:")
        print("\t"+str(self.grille[0:3]))
        print("\t"+str(self.grille[3:6]))
        print("\t"+str(self.grille[6:]))

In [3]:
exemple = Taquin([2, 8, 3, 1, 6, 4, 7, 0, 5])
exemple.affiche()

cp= 8
if exemple.estCoupValide(cp):
    print("Le coup ",cp, " est valide")
else:
    print("Le coup ",cp, " n'est pas valide")

nouvGrille = exemple.joueLeCoup(cp)
print("Position après avoir joué le coup :")
nouvGrille.affiche()
exemple.tousLesCoupsJouables()
print("Représentation compacte de cette grille : ")
print(exemple.toString())

Grille du taquin:
	[2, 8, 3]
	[1, 6, 4]
	[7, 0, 5]
('Le coup ', 8, " n'est pas valide")
Position après avoir joué le coup :
Grille du taquin:
	[2, 0, 3]
	[1, 6, 4]
	[7, 8, 5]
Représentation compacte de cette grille : 
283164705


### Position et coups
Lors de la recherche d'un chemin dans le graphe d'états (pour trouver une solution à une position de Taquin donnée), il est nécessaire de pouvoir associer à une position la liste des coups qui ont permis d'atteindre cette position depuis la position de Taquin initiale.

On représente une telle position par la classe suivante.

In [4]:
from collections import deque

class PositionTaquin:
    """ une position est une grille de Taquin ainsi qu'une liste de coups pour 
        l'atteindre depuis la position initiale
    """
    def __init__(self,taq):
        """ taq: le Taquin (contient la position)
        """
        self.taquin = taq
        self.moveList = []   # pour stocker les coups joués pour atteindre cette valeur
        
    def addListMove(self,oneListMove):
        """ ajout d'un coup à la position
            OneListMove: liste de coups
        """
        for c in oneListMove:
            self.moveList.append(c)            
        
    def getListMove(self):
        """ rend la liste des coups pour atteindre ce Taquin            
        """
        return self.moveList
    
    def getTaquin(self):
        """ rend le Taquin
        """
        return self.taquin
    
    def affiche(self):
        """ Pour voir cette position et la liste des coups
        """
        self.taquin.affiche()
        print("Suite de coups pour l'atteindre: ")
        for e in self.moveList:
            print(e," ")
    

### Recherche aléatoire
En utilisant les classes précédentes, écrire la fonction *rechercheAleatoire()* qui prend en argument un objet de la classe Taquin (position initiale de la recherche) et rend la liste des coups à jouer depuis cette position initiale pour atteindre la position finale du jeu. Si la recherche ne trouve aucune solution pour atteindre la position finale, la fonction rend None.
La liste des coups est trouvée en appliquant une recherche aléatoire sur le graphe des états du jeu.

In [40]:
from random import randint, shuffle
def rechercheAleatoire(taquin):
    ens = []
    pile = deque()
    taquin = PositionTaquin(taquin)
    while(taquin.getTaquin().estFinal() == False):
        coupPossible = taquin.getTaquin().tousLesCoupsJouables()
        prochainTaquin = []
        listeCoup = []
        for coup in coupPossible:
            newTaquin = taquin.getTaquin().joueLeCoup(coup)
            if(newTaquin.toString() not in ens):
                prochainTaquin.append(newTaquin)
                listeCoup.append(coup)
        if(len(prochainTaquin) == 0):
            if(len(pile) == 0):
                print("On a pas trouvé de solution")
                return None
        else:
            pile.append(taquin)
            indiceCoup = randint(0, len(listeCoup) - 1)
            newTaquin = PositionTaquin(prochainTaquin[indiceCoup])
            newTaquin.addListMove(taquin.getListMove() + [listeCoup[indiceCoup]])
            pile.append(newTaquin)
        ens.append(taquin.getTaquin().toString())
        shuffle(pile)
        taquin = pile.pop()
    return taquin

In [86]:
exemple = Taquin([2, 8, 3, 1, 6, 4, 7, 0, 5])
print("Grille de Taquin initiale :")
exemple.affiche()
print("Coups à jouer pour résoudre ce Taquin :")
rechercheAleatoire(exemple).affiche()

Grille de Taquin initiale :
Grille du taquin:
	[2, 8, 3]
	[1, 6, 4]
	[7, 0, 5]
Coups à jouer pour résoudre ce Taquin :
Grille du taquin:
	[1, 2, 3]
	[8, 0, 4]
	[7, 6, 5]
Suite de coups pour l'atteindre: 
(6, ' ')
(1, ' ')
(2, ' ')
(8, ' ')
(1, ' ')
(2, ' ')
(8, ' ')
(1, ' ')
(2, ' ')


### Recherche en profondeur

Ecrire la fonction *rechercheProfondeur()* qui prend en argument un objet de la classe Taquin (position initiale de la recherche) et rend la liste des coups à jouer depuis cette position initiale pour atteindre la position finale du jeu. La liste des coups est trouvée en appliquant une recherche en profondeur sur le graphe des états du jeu.
Si la recherche ne trouve aucune solution pour atteindre la position finale, la fonction rend None.


In [51]:
def rechercheProfondeur(taquin):
    taquin = PositionTaquin(taquin)
    coupPossible = taquin.getTaquin().tousLesCoupsJouables(

SyntaxError: invalid syntax (<ipython-input-51-930ec6c83d87>, line 3)

### Recherche en largeur

Ecrire la fonction *rechercheLargeur()* qui prend en argument un objet de la classe Taquin (position initiale de la recherche) et rend la liste des coups à jouer depuis cette position initiale pour atteindre la position finale du jeu. La liste des coups est trouvée en appliquant une recherche en largeur sur le graphe des états du jeu.
Si la recherche ne trouve aucune solution pour atteindre la position finale, la fonction rend None.



In [40]:
def rechLargeur(taquin):
    ens = []
    File = deque()
    taquin = PositionTaquin(taquin)
    File.append(taquin)
    i = 0
    while(len(File) != 0):
        i+= 1
        x = File.popleft()
        if(x.getTaquin().estFinal()):
            return x
        for coup in x.getTaquin().tousLesCoupsJouables():
            y = x.getTaquin().joueLeCoup(coup)
            if(y.toString() not in ens):
                ens.append(y.toString)
                y = PositionTaquin(y)
                y.addListMove(x.getListMove() + [coup])
                File.append(y)
    print(i)

In [41]:
exemple = Taquin([2, 8, 3, 1, 6, 4, 7, 0, 5])
print("Grille de Taquin initiale :")
exemple.affiche()
print("Coups à jouer pour résoudre ce Taquin :")
rechLargeur(exemple).affiche()

Grille de Taquin initiale :
Grille du taquin:
	[2, 8, 3]
	[1, 6, 4]
	[7, 0, 5]
Coups à jouer pour résoudre ce Taquin :
Grille du taquin:
	[1, 2, 3]
	[8, 0, 4]
	[7, 6, 5]
Suite de coups pour l'atteindre: 
(6, ' ')
(8, ' ')
(2, ' ')
(1, ' ')
(8, ' ')


### Recherche par l'algorithme A$*$

Afin de pouvoir appliquer l'algorithme A$*$, il est nécessaire d'utiliser une heuristique. 

Ecrire la fonction *evalH()* qui, pour une grille de Taquin donnée, rend le nombre de cases non nulles mal placées dans cette grille.

In [43]:
def evalH(taquin):
    correcte = [1,2,3,8,0,4,7,6,5]
    ko = 0
    for i in range(len(taquin.grille)):
        if(taquin.grille[i] != correcte[i] and taquin.grille[i] != 0):
            ko += 1
    return ko

En vous aidant de la classe *PositionTaquin*, écrire la classe *PositionTaquinEvaluee* qui contient aussi les évaluations *g(e)* et *h(e)* pour une grille de Taquin *e*.

In [46]:
from collections import deque

class PositionTaquinEvaluee:
    """ une position est une grille de Taquin ainsi qu'une liste de coups pour 
        l'atteindre depuis la position initiale
    """
    def __init__(self,taq, g):
        """ taq: le Taquin (contient la position)
        """
        self.taquin = taq
        self.moveList = []  
        self.h = evalH(self.taquin)
        self.g = g

    def addListMove(self,oneListMove):
        """ ajout d'un coup à la position
            OneListMove: liste de coups
        """
        for c in oneListMove:
            self.moveList.append(c)            
        
    def getListMove(self):
        """ rend la liste des coups pour atteindre ce Taquin            
        """
        return self.moveList
    
    def getTaquin(self):
        """ rend le Taquin
        """
        return self.taquin
    
    def affiche(self):
        """ Pour voir cette position et la liste des coups
        """
        self.taquin.affiche()
        print("Suite de coups pour l'atteindre: ")
        for e in self.moveList:
            print(e," ")

Ecrire la fonction *rechercheAEtoile()* qui prend en argument un objet de la classe Taquin (position initiale de la recherche) et rend la liste des coups à jouer depuis cette position initiale pour atteindre la position finale du jeu. La liste des coups est trouvée en appliquant l'algorithme A$*$ sur le graphe des états du jeu.
Si la recherche ne trouve aucune solution pour atteindre la position finale, la fonction rend None.




In [47]:
def rechercheAEtoile(taquin):
    etat = PositionTaquinEvaluee(taquin, 0)
    ens = []
    while(etat.getTaquin().estFinal() == False):
        for coup in etat.getTaquin().tousLesCoupsJouables():
            newEtat = etat.getTaquin().joueLeCoup(coup)
            newEtat = PositionTaquinEvaluee(newEtat, etat.g + 1)
            newEtat.addListMove(etat.getListMove() + [coup])
            ens.append((newEtat.g + newEtat.h, newEtat))
        coupleMin = min(ens)
        ens.remove(coupleMin)
        etat = coupleMin[1]
    return etat

In [48]:
exemple = Taquin([2, 8, 3, 1, 6, 4, 7, 0, 5])
print("Grille de Taquin initiale :")
exemple.affiche()
print("Coups à jouer pour résoudre ce Taquin :")
rechercheAEtoile(exemple).affiche()

Grille de Taquin initiale :
Grille du taquin:
	[2, 8, 3]
	[1, 6, 4]
	[7, 0, 5]
Coups à jouer pour résoudre ce Taquin :
Grille du taquin:
	[1, 2, 3]
	[8, 0, 4]
	[7, 6, 5]
Suite de coups pour l'atteindre: 
(6, ' ')
(8, ' ')
(2, ' ')
(1, ' ')
(8, ' ')


## Etape 2: Jeu de Taquin plus complexe

En vous aidant de la description du jeu (https://fr.wikipedia.org/wiki/Taquin) écrire une fonction *rechercheAEtoile()* qui permette de trouver la solution d'une position de Taquin $4\times 4$.

In [87]:
class Taquin4:
    def __init__(self,grille):
        """ grille: liste de 9 entiers de 0 à 9
            Hypothese: la grille donnée est correcte (il faudrait faire une vérification...)
        """
        self.grille = grille
    
    def estFinal(self):
        """ rend True si c'est une position finale du Taquin"""
        reference = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]
        return self.grille == reference
    
    def estCoupValide(self,coup):
        """ coup: entier de 1 à 9
            -> pour vérifier qu'un coup peut être jouer avant de le jouer
        """
        if coup == 0:
            # on ne joue pas la case vide !
            return False
        # le coup est valide si on peut déplacer la valeur vers la case 0
        indexCoup = self.grille.index(coup)
        indexZero = self.grille.index(0)
        # On vérifie si les cases sont à côté
        coordCoup = (int(indexCoup / 4.0), indexCoup % 4)  # (numLigne, numColonne)
        coordZero = (int(indexZero / 4.0), indexZero % 4)  # (numLigne, numColonne)
        if (coordCoup[0] == coordZero[0]):  
            # déplacement sur même ligne
            return (coordCoup[1] == coordZero[1]+1) or (coordCoup[1] == coordZero[1]-1)
        elif (coordCoup[1] == coordZero[1]): 
            # déplacement sur une même colonne
            return (coordCoup[0] == coordZero[0]+1) or (coordCoup[0] == coordZero[0]-1)
        
    def tousLesCoupsJouables(self):
        """ rend la liste des jouables (coups valides donc) dans la position 
            de ce taquin
        """
        return [i for i in range(1,16) if self.estCoupValide(i)]
                    
    def joueLeCoup(self,coup):
        """ coup: entier de 1 à 9 qui correspond à la valeur faut déplacer donc
            Hypothese: le coup est valide
            -> pour jouer le coup on échange l'entier donné avec le chiffre 0 (case vide)
            La fonction renvoie un nouveau Taquin (le Taquin courant n'est donc pas modifié)
        """
        indexCoup = self.grille.index(coup)
        coordCoup = (int(indexCoup / 4.0), indexCoup % 4)  # (numLigne, numColonne)
        indexZero = self.grille.index(0)
        coordZero = (int(indexZero / 4.0), indexZero % 4)  # (numLigne, numColonne)
        resultat = []
        for i in range(0,len(self.grille)):
            if i == indexCoup:
                resultat.append(0)
            elif i == indexZero:
                resultat.append(coup)
            else:
                resultat.append(self.grille[i])
        grilleResultat = Taquin4(resultat)
        return grilleResultat
        
    def toString(self):
        """ Conversion de la grille en une chaîne compacte
            Cette chaîne sert à pouvoir mettre des taquins dans un set() ou un dict()
        """
        return ''.join(str(e) for e in self.grille)
        
    def affiche(self):
        """ Affichage simple du taquin """
        print("Grille du taquin:")
        print("\t"+str(self.grille[0:4]))
        print("\t"+str(self.grille[4:8]))
        print("\t"+str(self.grille[8:12]))
        print("\t"+str(self.grille[12:]))
              